In [ ]:
from skimage import io
import skimage.filters as filt
from matplotlib import pyplot as plt
import numpy as np
from skimage.morphology import skeletonize, medial_axis, binary_closing
from scipy.interpolate import interp1d
import scipy.ndimage
from scipy.optimize import curve_fit
from scipy.signal import find_peaks
from scipy.stats import wilcoxon, ttest_rel,shapiro
from scipy.stats import norm
import scipy.stats as stats
from scipy.ndimage import gaussian_filter
from os import listdir
from os.path import isfile, join
from tqdm import trange, tqdm
from skimage import feature
from skimage.filters import meijering, sato, frangi, hessian
from scipy.stats import mannwhitneyu

In [ ]:
def Shift3(T,show,ruffle):
    MEM=np.max(Membrane,0)
    img=meijering(MEM,**kwargs)
    thresh = filt.threshold_otsu(img)
    img=img>thresh
    skeleton=skeletonize(img)
    skeleton=skeleton*MASK

    if ruffle==1:
        x,y=np.where(img*MASK==1)
    else:
        x,y=np.where(MASK-img==1)

    X=np.linspace(-0.8,1.8,200)
    countT=0
    MplotT=[]
    SplotT=[]

    countB=0
    MplotB=[]
    SplotB=[]
    HB=[]
    HT=[]
    for i in range(len(x)):
        test=0
        zlineM=Membrane[:80,x[i],y[i]]
        zlineS=SNX[:80,x[i],y[i]]
        zlineM=zlineM-np.min(zlineM)
        zlineM=zlineM/np.max(zlineM)

        zlineS=zlineS-np.min(zlineS)
        zlineS=zlineS/np.max(zlineS)

        peaks,_=find_peaks(zlineM,height=0.2,distance=8,prominence=0.15)

        if len(peaks)==2:
            fm=interp1d((np.arange(len(zlineM))-peaks[0])/(peaks[1]-peaks[0]),zlineM)
            fs=interp1d((np.arange(len(zlineM))-peaks[0])/(peaks[1]-peaks[0]),zlineS)


            try:
                Ym=fm(X)
            except:
                test=1

            try:
                Ys=fs(X)
            except:
                test=1

            if test==0:
                cut=100
                if(np.sum(Ym[:cut]*Ys[:cut])**2/np.sum(Ym[:cut]*Ym[:cut])/np.sum(Ys[:cut]*Ys[:cut])>T):
                    HB.append(peaks[1]-peaks[0])
                    countB+=1
                    MplotB.append(Ym[:cut]/max(Ym[:cut]))
                    SplotB.append(Ys[:cut]/max(Ys[:cut]))
                    if show==1:
                        plt.plot(X[:cut],Ym[:cut]/np.max(Ym[:cut]),'r--')
                        plt.plot(X[:cut],Ys[:cut]/np.max(Ys[:cut]),'g--')
                        plt.plot(X[:cut],X[:cut]*0+0.2,'k--')
                        plt.pause(0.1)
                if(np.sum(Ym[cut:]*Ys[cut:])**2/np.sum(Ym[cut:]*Ym[cut:])/np.sum(Ys[cut:]*Ys[cut:])>T):
                    HT.append(peaks[1]-peaks[0])
                    countT+=1
                    MplotT.append(Ym[cut:]/max(Ym[cut:]))
                    SplotT.append(Ys[cut:]/max(Ys[cut:]))
                    if show==1:
                        plt.plot(X[cut:],Ym[cut:]/np.max(Ym[cut:]),'r--')
                        plt.plot(X[cut:],Ys[cut:]/np.max(Ys[cut:]),'g--')
                        plt.plot(X[cut:],X[cut:]*0+0.2,'k--')
                        plt.pause(0.1)
    
    print('Bottom:'+str(countB)+'/'+str(len(x)))
    print('Top:'+str(countB)+'/'+str(len(x)))
    
    return MplotT,SplotT,MplotB,SplotB,HB,HT


def find_treshR(X):
    X=X/max(X)
    ind=len(X)-1
    STOP=0
    while STOP==0:
        if X[ind]>0.2:
            STOP=1
        else:
            ind-=1
    f=interp1d(X[ind-1:ind+2],[ind-1,ind,ind+1])
    ind=f(0.2)
    return ind

def find_treshL(X):
    #plt.plot(X)
    #plt.pause(0.1)
    X=X/max(X)
    ind=0
    STOP=0
    while STOP==0:
        if X[ind]>0.2:
            STOP=1
        else:
            ind+=1
    f=interp1d(X[ind-1:ind+2],[ind-1,ind,ind+1])
    ind=f(0.2)
    return ind

def Bootstrap(X0,X1,Y1):
    X0=np.array(X0)
    X1=np.array(X1)
    Y1=np.array(Y1)
    out=[]
    for i in range(100):
        if np.sum(X0==i)>0:
            p=np.random.permutation(np.sum(X1==i))
            out.append(Y1[X1==i,:][p,:][:np.sum(X0==i),:])
            
    return np.array(FLAT(out))

def FLAT(X):
    return [item for sublist in X for item in sublist]

def PLOT(X0,X1,X2,X3):
    plt.figure(figsize=[5,5])
    plt.fill_between([-1,100],[0,0],[-1,-1],color='k',alpha=0.1)
    plt.violinplot(X0,positions=[0],showextrema=False,showmeans=True,points=100)
    plt.violinplot(X1,positions=[1],showextrema=False,showmeans=True,points=100)
    plt.violinplot(X2,positions=[2],showextrema=False,showmeans=True,points=100)
    plt.violinplot(X3,positions=[3],showextrema=False,showmeans=True,points=100)

    plt.xticks([0,1,2,3],['Cell\nbottom\n(ruffle)','Cell\ntop\n(ruffle)','Cell\nbottom\n(flat)','Cell\ntop\n(flat)'])
    plt.ylabel('SNX localisation relative to membrane edge')
    plt.ylim(-0.5,0.6)
    plt.xlim(-0.5,3.5)

    plt.plot([0,1],[0.4,0.4],'k')
    if np.round(Ztest(X0,X1),3)==0:
        plt.text(0.20,0.42,'p<0.001')
    else:
        plt.text(0.20,0.42,'p='+str(np.round(Ztest(X0,X1),3)))

    plt.plot([2,3],[0.4,0.4],'k')
    if np.round(Ztest(X2,X3),3)==0:
        plt.text(2.20,0.42,'p<0.001')
    else:
        plt.text(2.20,0.42,'p='+str(np.round(Ztest(X2,X3),3)))


    plt.plot([1,3],[0.3,0.3],'k')
    if np.round(Ztest(X1,X3),3)==0:
        plt.text(1.70,0.32,'p<0.001')
    else:
        plt.text(1.70,0.32,'p='+str(np.round(Ztest(X1,X3),3)))

    plt.plot([0,2],[0.5,0.5],'k')
    plt.text(0.70,0.52,'p='+str(np.round(Ztest(X0,X2),3)))
    
def Ztest(X,Y):
    Zscore=(np.mean(X)-np.mean(Y))/np.sqrt(np.std(X)**2+np.std(Y)**2)
    return norm.sf(abs(Zscore))*2

In [ ]:
TR=[]
TF=[]
BR=[]
BF=[]

X=np.linspace(-0.8,1.8,200)

In [ ]:
mypath= #put path to the fluorescent image image
kwargs = {'sigmas': [2], 'mode': 'reflect', 'black_ridges':0}
mask = io.imread(mypath)
mypath= #put path to segmentation of the cell leading edge
MASK_FULL = io.imread(mypath)

MT=[]
ST=[]
MB=[]
SB=[]
HB_A=[]
HT_A=[]

MT2=[]
ST2=[]
MB2=[]
SB2=[]
HB_A2=[]
HT_A2=[]

for iii in range(0,MASK_FULL.shape[0]):
    if np.sum(MASK_FULL[iii,:,:])<MASK_FULL[iii,:,:].shape[0]*MASK_FULL[iii,:,:].shape[0]/3:
        print(iii+1)
        MASK=MASK_FULL[iii,:,:]
        Membrane=mask[iii,:,0,:,:]
        SNX=mask[iii,:,1,:,:]
        MplotT,SplotT,MplotB,SplotB,HB,HT=Shift3(0.3,0,1)
        MT.append(MplotT)
        ST.append(SplotT)
        MB.append(MplotB)
        SB.append(SplotB)
        HB_A.append(HB)
        HT_A.append(HT)
        
        
        MplotT,SplotT,MplotB,SplotB,HB,HT=Shift3(0.3,0,0)
        MT2.append(MplotT)
        ST2.append(SplotT)
        MB2.append(MplotB)
        SB2.append(SplotB)
        HB_A2.append(HB)
        HT_A2.append(HT)
                    
        
        
HT_A=FLAT(HT_A)
HT_A2=FLAT(HT_A2)
HB_A=FLAT(HB_A)
HB_A2=FLAT(HB_A2)

MT=FLAT(MT)
ST=FLAT(ST)
MB=FLAT(MB)
SB=FLAT(SB)

MT2=FLAT(MT2)
ST2=FLAT(ST2)
MB2=FLAT(MB2)
SB2=FLAT(SB2)

SBot=np.zeros(1000)
STop=np.zeros(1000)

for i in trange(len(SBot)):
    p=np.random.permutation(len(SB))
    SBot[i]=-(X[1]-X[0])*(find_treshL(np.mean(np.array(SB)[p,:][:100,:],0))-find_treshL(np.mean(np.array(MB)[p,:][:100,:],0)))
    STop[i]=(X[1]-X[0])*(find_treshR(np.mean(np.array(ST)[p,:][:100,:],0))-find_treshR(np.mean(np.array(MT)[p,:][:100,:],0)))
    
SBot2=np.zeros(1000)
STop2=np.zeros(1000)

for i in trange(len(SBot2)):
    p=np.random.permutation(len(SB))
    SBot2[i]=-(X[1]-X[0])*(find_treshL(np.mean(Bootstrap(HT_A,HB_A2,SB2)[p,:][:100,:],0))-find_treshL(np.mean(Bootstrap(HT_A,HB_A2,MB2)[p,:][:100,:],0)))
    STop2[i]=(X[1]-X[0])*(find_treshR(np.mean(Bootstrap(HT_A,HT_A2,ST2)[p,:][:100,:],0))-find_treshR(np.mean(Bootstrap(HT_A,HT_A2,MT2)[p,:][:100,:],0)))
    
PLOT(SBot,STop,SBot2,STop2)

TR.append(np.mean(STop))
TF.append(np.mean(STop2))
BR.append(np.mean(SBot))
BF.append(np.mean(SBot2))

In [ ]:
jit=(np.random.rand(len(BR))-0.5)/4

plt.figure(figsize=[2.5,5])
for i in range(len(BR)):
    plt.plot(np.arange(0,2)+jit[i],[TF[i],TR[i]],'k')
    

plt.plot(np.zeros(len(BR))+jit+1,np.array(TR),'o')
plt.plot(np.zeros(len(BR))+jit,np.array(TF),'o')
plt.ylim(-0.3,0.1)
plt.xlim(-0.5,1.5)

plt.xticks([0,1],['Cell\ntop\n(flat)','Cell\ntop\n(ruffle)'])
plt.ylabel('SNX localisation [normalised]')

ttest_rel(TR,TF)

plt.plot([0,1],[0.01,0.01],'k')
plt.text(0.2,0.02,'p='+str(np.round(ttest_rel(TR,TF)[1],3)))

In [ ]:
plt.figure(figsize=[2.5,5])
#plt.fill_between([-1,100],[0,0],[-1,-1],color='k',alpha=0.1)
for i in range(len(BR)):
    plt.plot(np.arange(0,2),[-BF[i],-BR[i]],'k')
    
plt.plot(np.zeros(len(BR))+1,-np.array(BR),'d')
plt.plot(np.zeros(len(BR)),-np.array(BF),'d')
plt.ylim(-0.15,0.2)
plt.xlim(-0.5,1.5)

plt.xticks([0,1],['Cell\nbottom\n(flat)','Cell\nbottom\n(ruffle)'])
plt.ylabel('SNX localisation relative to membrane edge')

plt.plot([0,1],[0.16,0.16],'k')
plt.text(0.2,0.17,'p='+str(np.round(ttest_rel(BF,BR)[1],3)))